In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\akg96\\Desktop\\Github\\Text-summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\akg96\\Desktop\\Github\\Text-summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
          
            config = self.config.model_trainer
            params = self.params.TrainingArguments

            create_directories([config.root_dir])

            model_trainer_config = ModelTrainerConfig(
                root_dir = config.root_dir,
                data_path = config.data_path,
                model_ckpt = config.model_ckpt,
                num_train_epochs =  params.num_train_epochs,
                warmup_steps= params.num_train_epochs,
                per_device_train_batch_size = params.per_device_train_batch_size, 
                weight_decay = params.weight_decay,
                logging_steps = params.logging_steps,
                evaluation_strategy = params.evaluation_strategy,
                eval_steps = params.eval_steps,
                save_steps = params.save_steps,
                gradient_accumulation_steps  = params.gradient_accumulation_steps
          )

            return model_trainer_config

In [9]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\akg96\anaconda3\envs\Text-Summarizer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):

        self.config = config

    def train(self):

        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        print(self.config)

        trainer_args = TrainingArguments(
            output_dir = self.config.root_dir,
            num_train_epochs =  int(self.config.num_train_epochs),
            warmup_steps= int(self.config.num_train_epochs),
            per_device_train_batch_size = int(self.config.per_device_train_batch_size), 
            weight_decay = float(self.config.weight_decay),
            logging_steps = int(self.config.logging_steps),
            evaluation_strategy = self.config.evaluation_strategy,
            eval_steps = int(self.config.eval_steps),
            save_steps =  float(self.config.save_steps),
            gradient_accumulation_steps  = int(self.config.gradient_accumulation_steps)
        )

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                          tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                          train_dataset = dataset_samsum_pt["test"],
                          eval_dataset = dataset_samsum_pt["validation"])
        
        trainer.train()

        # save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        # Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [11]:
try: 
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()

except Exception as e:
    raise e

<_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'>
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'url = "https://drive.google.com/file/d/15SQdQy-TkE4A68e-hLX_4WOtzsOU7dMw/view?usp=sharing"', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'ALL_REQUIRED_FILES': ['train', 'test', 'validation']}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'data_path': 'artifacts/data_transformation/samsum_dataset', 'model_ckpt': 'google/pegasus-cnn_dailymail'}}
[2023-09-30 14:02:37,467: INFO: common: yaml file: config\config.yaml loaded successfully]
<_io.TextIOWrapper name='p

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ModelTrainerConfig(root_dir='artifacts/model_trainer', data_path='artifacts/data_transformation/samsum_dataset', model_ckpt='google/pegasus-cnn_dailymail', num_train_epochs=1, warmup_steps=1, per_device_train_batch_size=1, weight_decay=0.01, logging_steps=10, evaluation_strategy='steps', eval_steps=500, save_steps='1e6', gradient_accumulation_steps=16)


 20%|█▉        | 10/51 [22:29<1:30:37, 132.62s/it]

{'loss': 3.1268, 'learning_rate': 4.1e-05, 'epoch': 0.2}


...

{'loss': 2.2922, 'learning_rate': 1.1000000000000001e-05, 'epoch': 0.78}


 98%|█████████▊| 50/51 [13:14:51<02:49, 169.20s/it]   

{'loss': 2.1373, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.98}


100%|██████████| 51/51 [13:15:49<00:00, 936.26s/it]


{'train_runtime': 47749.5021, 'train_samples_per_second': 0.017, 'train_steps_per_second': 0.001, 'train_loss': 2.4446659976360845, 'epoch': 1.0}


In [12]:
# !pip install accelerate -U/